In [3]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

2


In [4]:
# set up a static tweezer array
frequency_spacing = 1.e6
n_tweezers = 2
f_list = np.arange(75.e6 - n_tweezers*frequency_spacing/2,75.e6 + n_tweezers*frequency_spacing/2, frequency_spacing)
print(f_list)
a_list = [.1,.1]

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()

[74000000. 75000000.]


In [6]:
# fm the tweezer array for some amount of time

# how much time to fm?
fm_time = 1.

modulation_freq  = 1.
modulation_depth = 2.e6

# rate at which to update frequency (sample rate)
period_s = 20.e-3

# number of samples in a period
num_samples = int(1/(period_s * modulation_freq))

# list of sample numbers
sample_range = np.arange(num_samples)*period_s

# empty array to be filled with frequency values for each dds core
freq_lists = np.empty([len(f_list),len(sample_range)])

# compute and fill in the above array
for i in range(len(f_list)):
    freq_lists[i] = f_list[i] + modulation_depth * np.sin(2*np.pi*sample_range*modulation_freq)

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(period_s)
dds.exec_at_trg()
dds.write_to_card()

# set trigger count to 0, for keeping track of how much time has passed once in trg_timer mode
trg_count_init = trigger.trigger_counter()

# pre_fill the buffer
fill_max = dds.queue_cmd_max()
counter = 0
for counter in range(fill_max // 2 - 4):
    for tone_idx in range(np.shape(freq_lists)[0]):
        freq_Hz = freq_lists[tone_idx][counter % num_samples]
        dds[tone_idx].freq(freq_Hz)
    dds.exec_at_trg()
dds.write_to_card()

fill_count = dds.queue_cmd_count()
print("Pre-fill buffer: {}/{}".format(fill_count, fill_max))

# start streaming
fill_number = fill_max // 4
fill_check = fill_max - fill_number

trigger.force()

# how many trigger events have happened already?
how_many_triggers = float(trigger.trigger_counter() - trg_count_init)

while how_many_triggers < (fm_time / period_s):
    how_many_triggers = float(trigger.trigger_counter() - trg_count_init)
    print(how_many_triggers)
    while True:
        fill_count = dds.queue_cmd_count()
        if fill_count < fill_check: break
    print("Adding a block of commands to buffer")
    for i in range(fill_number // 2):
        for tone_idx in range(np.shape(freq_lists)[0]):
            freq_Hz = freq_lists[tone_idx][counter % num_samples]
            dds[tone_idx].freq(freq_Hz)
            dds.exec_at_trg()
            counter += 1
            counter %= num_samples
    dds.write_to_card()
    status = dds.status()

    # check that the queue hasn't underrun
    if status & spcm.SPCM_DDS_STAT_QUEUE_UNDERRUN:
        break

# reset trigger mode to external at the end of the fm time
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()


Pre-fill buffer: 4088/4096
1.0


Exception ignored in: <function Device.__del__ at 0x00000223D00512D0>
Traceback (most recent call last):
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 78, in __del__
    self.stop()
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 321, in stop
    self.cmd(M2CMD_CARD_STOP, *args)
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 274, in cmd
    self.set_i(SPC_M2CMD, cmd)
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 440, in set_i
    self._check_error(spcm_dwSetParam_i64(self._handle, register, value))
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\pyspcm.py", line 337, in spcm_dwSetParam_i64
    return spcm_dwSetParam_i64_ (hDrv, lReg, llVal)
OSError: [WinError -1073741816] Windows Error 0xc0000008


KeyboardInterrupt: 

In [ ]:
trigger.force()

In [7]:
# card.close(card._handle)
card.stop()

card.close(card._handle)